In [1]:
import pandas as pd 
import time
from time import sleep
from datetime import date, timedelta
import os
from os import listdir
from os.path import isfile, join
from os import path
from pathlib import Path
import fitz  # PyMuPDF
import matplotlib.pyplot as plt
import re
import cv2
from PIL import Image

In [ ]:
root = Path('/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis')
textCR_path = os.path.join(root,"input/Text_CR")

In [45]:
textCR_path = os.path.join(root,"input/Text_CR" )
text_path = os.path.join(textCR_path,"29-11-1979.txt")
text_page = open("/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Text_CR/29-11-1979.txt", "r")
text_page = text_page.read()

In [49]:
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", 
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", 
    "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
    "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
    "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming"
]

# Join the states into a single regex pattern
state_pattern = "|".join(states)

In [53]:
def process_transcript(final_text):
    #Define the pattern to identify speakers
    speaker_pattern = rf'(Mr\. [A-Z]{{3,}}\.|Mr\. [A-Z]{{3,}} of (?:{state_pattern})\.|The SPEAKER\. |The VICE PRESIDENT. |The PRESIDING OFFICER )'

    #Split the final text 
    split_text = re.split(speaker_pattern,final_text)

    #Initialize an empty list to store the results :))
    data = []

    # Create a list of current speakers and current speeches to match them
    current_speaker = None
    current_speech = []

    for segment in split_text :
        if re.match(speaker_pattern, segment):
            # If the segment in the split segment is a speaker, save the previous´s speech if any
            if current_speaker and current_speech:
                data.append({"speaker": current_speaker, "speech": " ".join(current_speech).strip()}) # Strip trims the strings
            
            # Update curent speaker and reset current speech
            current_speaker = segment.strip()
            current_speech = []

        else:
            #Append the sempent to the current speech if the segment was not a speaker, but a speech
            current_speech.append(segment)
    
    #Add the last speakers's speech to the data
    if current_speaker and current_speech:
        data.append({"speaker": current_speaker, "speech": " ".join(current_speech).strip()})
    df = pd.DataFrame(data, columns=["speaker", "speech"])
    return df

In [59]:
import os
import glob
import pandas as pd  # Import pandas for DataFrame handling

# Define the path to the folder containing the text files
textCR_path = os.path.join(root,"input/Text_CR")

# Initialize an empty list to store all data_speech dictionaries
all_data_speeches = []

# Loop through each .txt file in the folder
for text_file in glob.glob(os.path.join(textCR_path, "*.txt")):
    # Open and read the text file
    with open(text_file, "r", encoding="utf-8") as file:
        text_page = file.read()

    # Process the text
    text_page = text_page.replace('-\n', "")  # Unify split words
    text_page = text_page.replace('\n', " ")  # Replace newlines with spaces
    text_page = text_page.replace("Mr,", "Mr.")  # Fix typo

    # Process the transcript (assuming process_transcript is defined elsewhere)
    data_speech = process_transcript(text_page)

    # Extract day, month, and year from the filename (assuming format: DD-MM-YYYY.txt)
    filename = os.path.basename(text_file)
    day, month, year = map(int, filename.split(".")[0].split("-"))

    # Add metadata to the data_speech dictionary
    data_speech["day"] = day
    data_speech["month"] = month
    data_speech["year"] = year
    data_speech["chamber"] = "upper"

    # Reorder columns
    data_speech = data_speech[['year', 'month', 'day', 'chamber', 'speaker', 'speech']]

    # Append the processed data_speech to the list
    all_data_speeches.append(data_speech)

# Combine all data_speech dictionaries into a single DataFrame
final_data_speech = pd.concat(all_data_speeches, ignore_index=True)

# Now `final_data_speech` contains all the processed data
print(final_data_speech)  # Replace this with your desired output logic

      year  month  day chamber                speaker  \
0     1979      9   13   upper          Mr. PACKWOOD.   
1     1979      9   13   upper            Mr. MUSKIE.   
2     1979      9   13   upper            Mr. MUSKIE.   
3     1979      9   13   upper          Mr. HATFIELD.   
4     1979      9   13   upper            Mr. MUSKIE.   
...    ...    ...  ...     ...                    ...   
9045  1979     10   17   upper            Mr. MORGAN.   
9046  1979     10   17   upper          Mr. PROXMIRE.   
9047  1979     10   17   upper          Mr. PROXMIRE.   
9048  1979     10   17   upper  The PRESIDING OFFICER   
9049  1979     10   17   upper  The PRESIDING OFFICER   

                                                 speech  
0     Mr. President, yield back my time.  ROUTINE MO...  
1     I ask unanimous con; sent that the order for t...  
2     Mr. President, I ask unanimous consent that Mi...  
3                 Mr. President, wil the Senator yield?  
4                        

In [64]:
csv_path = os.path.join(root,"input/database_CR/1979.csv")
final_data_speech.to_csv(csv_path)